# 製品プライサーは継続しました

その説明から、何かがどれだけの費用がかかるかを推定できるモデル。

##データキュレーションパート2

今日は、データセットをより大きなカバレッジに拡張し、トレーニング用の優れたデータセットに作成します。  
データキュレーションは、私たちが取り組んでいる他のことよりもエキサイティングではないように見えることがありますが、それはLLMエンジニアの責任の重要な部分であり、高品質のデータセットで独自の商用ソリューションを構築できるようにするための重要なクラフトです。

データセットはこちらです：  
https://huggingface.co/datasets/mcauley-lab/amazon-reviews-2023

すべての製品データセットを備えたフォルダーはこちらです。  
https://huggingface.co/datasets/mcauley-lab/amazon-reviews-2023/tree/main/raw/meta_categories

##重要なメモ - 最初に私を読んでください

複数の種類の製品をカバーする400,000アイテムの大規模なデータセットを作成しようとしています。 7週目には、このデータを使用して独自のモデルをトレーニングします。これはかなり大きなデータセットであり、選択したGPUによっては、トレーニングが20時間以上かかる場合があります。本当に楽しいでしょうが、計算ユニットに数ドルかかる可能性があります。

別の方法として、物事を迅速かつ低コストに保ちたい場合は、家電製品のみに焦点を当てた小さなデータセットを使用できます。同じ学習ポイントをカバーできます。結果は良好です - 完全なデータセットほど良くはありませんが、それでもかなり驚くべきことです！これをやりたい場合は、このフォルダーに「Lite.ipynb」と呼ばれる代替Jupyterノートブックを設定しました。

また、必要に応じて、最後のセルに保存するピクルスファイルをダウンロードして、このデータキュレーションをすべて実行しているのはショートカットです。ピクルスファイルは、https：//drive.google.com/drive/folders/1f_izgybvs9o0j5sb3xmtteqb3bxllzrwです

In [ ]:
# 輸入

import os
import random
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import numpy as np
import pickle

In [ ]:
# 環境

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Huggingfaceにログインします

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# HFログイン後のより多くのインポート

from loaders import ItemLoader
from items import Item

In [ ]:
%matplotlib inline

## アイテムローダーコード

Loaders.pyを見る - 私たちの生活を楽にするための便利なコードがいくつかあります

In [ ]:
# 前回と同じデータセットにロードします

items = ItemLoader("Home_and_Kitchen").load()

In [ ]:
# おなじみのアイテムを探してください。
print(items[1].prompt)

## 今、スケールアップします

大規模なホーム小売店で見つけるかもしれないすべてのアイテムのすべてのデータセットを見てみましょう - 電気、電子、オフィス、関連するが、衣服 /美しさ /本ではありません。

In [ ]:
dataset_names = [
    "Automotive",
    "Electronics",
    "Office_Products",
    "Tools_and_Home_Improvement",
    "Cell_Phones_and_Accessories",
    "Toys_and_Games",
    "Appliances",
    "Musical_Instruments",
]

In [ ]:
items = []
for dataset_name in dataset_names:
    loader = ItemLoader(dataset_name)
    items.extend(loader.load())

# さて、コーヒーブレイクの時間!!
# ちなみに、私は最大のデータセットを最初に置きます。それはより速くなります。

In [ ]:
print(f"A grand total of {len(items):,} items")

In [ ]:
# トークンカウントの分布を再度プロットします

tokens = [item.token_count for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="skyblue", bins=range(0, 300, 10))
plt.show()

In [ ]:
# 価格の分布をプロットします

prices = [item.price for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.1f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="blueviolet", bins=range(0, 1000, 10))
plt.show()

In [ ]:
category_counts = Counter()
for item in items:
    category_counts[item.category]+=1

categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# カテゴリ別のバーチャート
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="goldenrod")
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

plt.xticks(rotation=30, ha='right')

# 各バーの上に値ラベルを追加します
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# チャートを表示します
plt.show()

# 客観的

価格のバランスが取れているデータセットを作成します。安価なアイテムにはあまり重くなりませんでしたが、平均は60ドルを超えています。カテゴリのバランスを取るようにしてください - 自動車アイテムは少なくなります。

In [ ]:
# 各価格のキーが1ドルから999ドルのキーを持つDICTを作成します
# 価値で、その価格（最も近いラウンド番号まで）のアイテムのリストを入れてください

slots = defaultdict(list)
for item in items:
    slots[round(item.price)].append(item)

In [ ]:
# 価格の範囲からより均等に取ろうとする「サンプル」と呼ばれるデータセットを作成する
# 自動車以外のカテゴリからのアイテムにより多くの重みを与えます
# 再現性のためにランダムシードを設定します

np.random.seed(42)
random.seed(42)
sample = []
for i in range(1, 1000):
    slot = slots[i]
    if i>=240:
        sample.extend(slot)
    elif len(slot) <= 1200:
        sample.extend(slot)
    else:
        weights = np.array([1 if item.category=='Automotive' else 5 for item in slot])
        weights = weights / np.sum(weights)
        selected_indices = np.random.choice(len(slot), size=1200, replace=False, p=weights)
        selected = [slot[i] for i in selected_indices]
        sample.extend(selected)

print(f"There are {len(sample):,} items in the sample")

In [ ]:
# サンプルの価格の分布をプロットします

prices = [float(item.price) for item in sample]
plt.figure(figsize=(15, 10))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# わかりました、私たちは平均価格を上げて、スムーズな価格の人口を持っているという点でうまくいきました
# カテゴリを見てみましょう

category_counts = Counter()
for item in sample:
    category_counts[item.category]+=1

categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# バーチャートを作成します
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="lightgreen")

# チャートをカスタマイズします
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

plt.xticks(rotation=30, ha='right')

# 各バーの上に値ラベルを追加します
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# チャートを表示します
plt.show()

In [ ]:
# 自動車はまだリードしていますが、やや改善されました
# 別の視点については、パイを見てみましょう

plt.figure(figsize=(12, 10))
plt.pie(counts, labels=categories, autopct='%1.0f%%', startangle=90)

# 中央に円を追加してドーナツチャートを作成します（オプション）
centre_circle = plt.Circle((0,0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('Categories')

# 平等なアスペクト比は、パイが円として描かれることを保証します
plt.axis('equal')  

plt.show()

# データセットキュレーション！

優れたデータセットを作成しました。

いくつかの最終チェックをしましょう

In [ ]:
# プロンプトの文字カウントによって価格はどのように異なりますか？

sizes = [len(item.prompt) for item in sample]
prices = [item.price for item in sample]

# 散布図を作成します
plt.figure(figsize=(15, 8))
plt.scatter(sizes, prices, s=0.2, color="red")

# ラベルとタイトルを追加します
plt.xlabel('Size')
plt.ylabel('Price')
plt.title('Is there a simple correlation?')

# プロットを表示します
plt.show()

In [ ]:
def report(item):
    prompt = item.prompt
    tokens = Item.tokenizer.encode(item.prompt)
    print(prompt)
    print(tokens[-10:])
    print(Item.tokenizer.batch_decode(tokens[-10:]))

In [ ]:
report(sample[398000])

## 観察

Llamaトークナイザーの興味深い点は、GPT-4oで見たように、1〜999のすべての数値が1トークンにマッピングされることです。同じことは、すべての数字をトークンにマッピングするQWEN2、GEMMA、およびPHI3にも当てはまりません。これは、私たちのプロジェクトにとって少し便利であることが判明しますが、重要な要件ではありません。

＃ ついに

データをトレーニング、テスト、検証データセットに分割する時が来ました。

テスト目的でデータの5％〜10％を使用することが典型的ですが、実際にはこの時点で必要なものよりもはるかに多くあります。トレーニングには400,000ポイントを獲得し、テスト用に2,000ポイントを予約しますが、すべてを使用することはありません。


In [ ]:
random.seed(42)
random.shuffle(sample)
train = sample[:400_000]
test = sample[400_000:402_000]
print(f"Divided into a training set of {len(train):,} items and test set of {len(test):,} items")

In [ ]:
print(train[0].prompt)

In [ ]:
print(test[0].test_prompt())

In [ ]:
# 最初の250テストポイントで価格の分布をプロットする

prices = [float(item.price) for item in test[:250]]
plt.figure(figsize=(15, 6))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

# 最後に - 新しいデータセットをアップロードします

プロンプトに変換し、ハグFaceハブにアップロードします

In [ ]:
train_prompts = [item.prompt for item in train]
train_prices = [item.price for item in train]
test_prompts = [item.test_prompt() for item in test]
test_prices = [item.price for item in test]

In [ ]:
# リストからデータセットを作成します

train_dataset = Dataset.from_dict({"text": train_prompts, "price": train_prices})
test_dataset = Dataset.from_dict({"text": test_prompts, "price": test_prices})
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
# ハブにプッシュする準備ができていて、私の名前をHFユーザー名に置き換える準備ができている場合、これらの行の解除

# hf_user = "ed-donner"
# dataset_name = f "{hf_user}/pricer-data"
# dataset.push_to_hub（dataset_name、private = true）

In [ ]:
# もう一つ！
# 次回、このコードをすべて実行する必要がないように、トレーニングとテストデータセットを漬けてみましょう！

with open('train.pkl', 'wb') as file:
    pickle.dump(train, file)

with open('test.pkl', 'wb') as file:
    pickle.dump(test, file)

## あなたのためのトドス：

 - データセットをもっと調査してください！
 - トークンザーが3桁すべての価格を1つのトークンにトークン化することを確認します